# 第5单元: 使用PyTorch编写你的第一个深度强化学习算法: Reinforce. 并测试它的鲁棒性 💪

在这份笔记中, 你将从头编写你的第一个深度强化学习算法: Reinforce(也称为蒙特卡洛策略梯度).

Reinforce是一种**基于策略的方法**: 一种**尝试直接优化策略而不是使用动作价值函数**的深度强化学习算法. 更准确的说, Reinforce是策略梯度方法, 是基于策略方法的子类, 旨在**通过使用梯度提升(Gradient Ascent)估计最优策略的权重来直接优化策略.**

为了测试鲁棒性, 我们将在3个不同的简单环境进行训练:
* Cartpole-v1
* PongEnv
* PixelcopterEnv

❓如果你有任何问题, 请在discord的#study-group-unit5频道发帖 👉 https://discord.gg/aYka4Yhff9

🎮 环境:

* [CartPole-v1](https://www.gymlibrary.ml/environments/classic_control/cart_pole/)
* [PixelCopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)
* [Pong](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pong.html)

⬇️ 这是**你将在几分钟内实现的目标**的示例. ⬇️

![Sans titre.gif](./assets/Sans%20titre.gif)

## 这份笔记的目标🏆

在这份笔记学习结束后, 你将:

* 能够使用**PyTorch从头编写Reinforce算法.**
* 能够使用**简单的环境测试你的智能体的鲁棒性.**
* 能够通过精彩的回放和得分🔥**发布你训练的智能体到Hugging Face Hub.**

## 这份笔记来自深度强化学习课程
![Deep Reinforcement Learning Course.jpg](./assets/DeepReinforcementLearningCourse.jpg)

在这个免费课程中, 你将:

* 📖 研究深度强化学习的**理论和实践.**
* 🧑‍💻 学习**使用流行的深度强化学习库**, 例如Stable Baselines3, RL Baselines3 Zoo和RLlib.
* 🤖️ **在独特的环境中训练智能体.**

还有更多的课程 📚 内容 👉 https://github.com/huggingface/deep-rl-class

保持进度的最佳方式是加入我们的Discord服务器与社区和我们进行交流. 👉🏻 https://discord.gg/aYka4Yhff9

## 先决条件 🏗

在深入研究笔记之前, 你需要:

🔲 📚 [阅读第5单元的README.](https://github.com/huggingface/deep-rl-class/blob/main/unit5/README.md)

🔲 📚 通过阅读 👉 https://huggingface.co/blog/deep-rl-pg **学习策略梯度**

### 第0步: 设置GPU 💪

* 为了**更快的训练智能体, 我们将使用GPU,** 选择`修改 > 笔记本设置`
![image.png](./assets/image.png)

* `硬件加速器 > GPU`

![image.png](./assets/image1.png)

在笔记中, 我们需要生成一个回放视频. 因此在Colab(或你本地的jupyter)中, **我们需要一个虚拟屏幕能渲染环境**(记录视频帧).

下面的单元格将安装虚拟屏幕库并创建和运行虚拟屏幕. 🖥

In [ ]:
!apt install gitlfs ffmpeg
# 如果你使用IDE(例如PyCharm或VS Code)将不需要这些步骤.
!apt install python-opengl xvfb 
!pip install pyvirtualdisplay

In [ ]:
# 创建虚拟屏幕.
from pyvirtualdisplay import Display

virtual_display = Display(visible=False, size=(1400, 900))
virtual_display.start()

In [ ]:
import torch

In [13]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
print(device)